# CSC 620 HA #11

By: Mark Kim

Adapted from: [Utham Bathoju](https://www.kaggle.com/code/uthamkanth/beginner-tf-idf-and-cosine-similarity-from-scratch/notebook)

(I don't know why, but the links to different sections sometimes don't work)

This notebook is broken up into three sections as follows:

1. [Program_0](#program_0): Create a copy of the below notebook (or export it as
   python program), and add detailed description for each code block, in your
   own words.
2. [Program_1](#program_1): Revise the above program to replace the toy dataset
   with with a larger text dataset of your choice.
3. [Program_2](#program_2): Create a variation of Program_1 that uses only TF
   representation to compute the similarity between a query and document.
4. [Writeup](#Writeup): Submit a short write-up (1 to 2 paragraphs) that compares and contrasts the document rankings provided by Program_1 and Program_2 for the same 3 queries of your choice.  Reflect on why is the ranking different, which representation provides better ranking, etc.

## Program_0

Create a copy of the notebook and add detailed descriptions.

In [1]:
import math
import pandas as pd
import numpy as np

Define query and toy documents

In [2]:
#documents
doc1 = "I want to start learning to charge something in life"
doc2 = "reading something about life no one else knows"
doc3 = "Never stop learning"
#query string
query = "life learning"

## Raw term frequency counts

The following function was created simply to illustrate that term frequency is
not a good measure to base a search query from.  The terms have not been
normalized (e.g. capitalization removed, stopwords removed, etc.).  Furthermore,
we do not normalize to large term counts.  As we learned in lecture, a word that
appears 100 times should not weigh 100 times heavier in determining
"importance".  Lastly, for the data to be useful, we want to create
probabilities that terms appear, hence, raw counts will not be useful for us.

In [3]:
def compute_tf(docs_list):
    for doc in docs_list:
        doc1_lst = doc.split(" ")
        wordDict_1= dict.fromkeys(set(doc1_lst), 0)

        for token in doc1_lst:
            wordDict_1[token] +=  1
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
        
compute_tf([doc1, doc2, doc3])

         Document  in  learning  start  charge  something  to  life  want  I
0  Term Frequency   1         1      1       1          1   2     1     1  1
         Document  one  no  about  else  knows  reading  life  something
0  Term Frequency    1   1      1     1      1        1     1          1
         Document  Never  learning  stop
0  Term Frequency      1         1     1


## Normalized Term Frequency

The next functions attempt at converting the raw term frequency counts to some
sort of probability so that we can determine the probability a term appears in a
document.  Very basic normalization is done here, where the capitalization of
words is removed.

In [4]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf= dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    
        df.insert(loc=idx, column='Document', value=new_col)
        # print(df)
    return tf_doc

tf_doc = compute_normalizedtf([doc1, doc2, doc3])

# Inverse Document Frequency (IDF)

Here, we address the issue of large term counts having a disproportionate effect
in determining relevancy with respect to matching a particular query.  By
applying an inverse document frequency to the term frequency, we suppress the
weights of terms with high frequency counts.  Indeed, words that occur less
often are a better determinant of matching queries to a document.  Hence, we
apply the following formula to find the IDF:
$$ \operatorname{idf_t} = \log_{10}\left(\frac{N}{df_t}\right) $$
where $N$ is the total number of documents in the corpus and $df_t$ is the
number of documents in which the term $t$ appears.

In this case, it looks like the author increases the value for idf by $1$.  I am
not sure why the author does this since the formula doe not call for this
addition.  I have not removed the addition of $1$ because it does not change the
final results since this addition occurs in all cases.

In [5]:
def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in range (0, len(allDocuments)):
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
 
    if numDocumentsWithThisTerm > 0:
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0
    
def compute_idf(documents):
    idf_dict = {}
    for doc in documents:
        sentence = doc.split()
        for word in sentence:
            idf_dict[word] = inverseDocumentFrequency(word, documents)
    return idf_dict
    
idf_dict = compute_idf([doc1, doc2, doc3])

compute_idf([doc1, doc2, doc3])

{'I': 2.09861228866811,
 'want': 2.09861228866811,
 'to': 2.09861228866811,
 'start': 2.09861228866811,
 'learning': 1.4054651081081644,
 'charge': 2.09861228866811,
 'something': 1.4054651081081644,
 'in': 2.09861228866811,
 'life': 1.4054651081081644,
 'reading': 2.09861228866811,
 'about': 2.09861228866811,
 'no': 2.09861228866811,
 'one': 2.09861228866811,
 'else': 2.09861228866811,
 'knows': 2.09861228866811,
 'Never': 2.09861228866811,
 'stop': 2.09861228866811}

Using the IDF values above, the author then calculates the IDF score for each
word in the query.  The following function compares the query words to the IDF
dictionary and term frequency dictionary for each document to find the TF-IDF
score for each word in each document.

In [6]:
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents , query):
    tf_idf = []
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_doc[index]
        sentence = doc.split()
        for word in sentence:
            for text in query_tokens:
                if(text == word):
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf_idf , df
            
documents = [doc1, doc2, doc3]
tf_idf , df = compute_tfidf_with_alldocs(documents , query)
print(df)

   doc      life  learning
0  0.0  0.140547  0.140547
1  1.0  0.175683  0.000000
2  2.0  0.000000  0.468488


# Cosine Similarity

The author takes an incremental approach here to calculate cosine similarities
between documents here.  They first calculate a normalized term frequency
dictionary, followed by calculating an IDF dictionary.  Once the term frequency
and IDF are calculated a final document weight vector can be calculated from the
results.

### Term Frequency Function

In [7]:
#Normalized TF for the query string("life learning")
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

{'life': 0.5, 'learning': 0.5}


### IDF Function

In [8]:
#idf score for the query string("life learning")
def compute_query_idf(query):
    idf_dict_qry = {}
    sentence = query.split()
    documents = [doc1, doc2, doc3]
    for word in sentence:
        idf_dict_qry[word] = inverseDocumentFrequency(word ,documents)
    return idf_dict_qry
idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)

{'life': 1.4054651081081644, 'learning': 1.4054651081081644}


### TF-IDF Function

In [9]:
#tf-idf score for the query string("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query)
print(tfidf_dict_qry)

{'life': 0.7027325540540822, 'learning': 0.7027325540540822}


## Cosine Similarity Function

Finally, all the above results can be combined to calculate cosine similarity
with the following formula:
$$ \cos(\vec{q},\vec{d}) = \frac{\vec{q} \cdot \vec{d}}{\lVert\vec{q}\rVert
\lVert\vec{d}\rVert} =  \frac{\displaystyle\sum_{i=1}^{\lvert V\rvert} q_i
d_i}{\sqrt{\displaystyle\sum_{i=1}^{\lvert V\rvert}
q_i^2}\sqrt{\displaystyle\sum_{i=1}^{\lvert V\rvert} di^2}}$$

The `cosine_similarity` function is pretty self-explanatory.  It simply adds up
the products of the query and document TF-IDF scores, then divides them by the
product of the norms of each.

I am not sure why the original author created a generator for the flatten
function since we don't really need to use lazy programming here: we are looking
to calculate all results.

In [10]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
def cosine_similarity(tfidf_dict_qry, df , query , doc_num):
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()
   
    for keyword in tokens:
        dot_product += tfidf_dict_qry[keyword] * df[keyword][df['doc'] == doc_num]
        #||Query||
        qry_mod += tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        #||Document||
        doc_mod += df[keyword][df['doc'] == doc_num] * df[keyword][df['doc'] == doc_num]
    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    #implement formula
    denominator = qry_mod * doc_mod
    cos_sim = dot_product/denominator
     
    return cos_sim

from collections.abc import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                yield x
        else:        
             yield item


In [11]:
def rank_similarity_docs(data):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist())
    return cos_sim
similarity_docs = rank_similarity_docs(documents)
doc_names = ["Document1", "Document2", "Document3"]
print(doc_names)
print(list(flatten(similarity_docs)))

['Document1', 'Document2', 'Document3']
[1.0, 0.7071067811865475, 0.7071067811865475]


## Program_1

Load articles from The Onion, a satirical news outlet, then remove all rows that
contain invalid text.

In [12]:
import re
import pandas as pd

In [13]:
theonion = pd.read_csv("./file_archive/theonion.csv")
theonion = theonion.dropna()
theonion["processed"] = theonion["Content"].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))

Grab a small sample from the dataset to reduce computation time.

In [14]:
theonion = theonion.sample(frac=0.05, random_state=10)

In [15]:
theonion.head()

,Title,Published Time,Content,processed
537,Nuclear-Bomb Instructions Found In Pentagon,2001-12-12T15:00:00-06:00,"ARLINGTON, VA— In an alarming development, pla...",arlington va in an alarming development plans ...
4611,American Classmates Having Difficulty Understa...,2018-08-23T10:14:00-05:00,"SACRAMENTO, CA—Addressing the glaringly obviou...",sacramento caaddressing the glaringly obvious ...
1320,Dole Reveals One Cantaloupe Out There Contains...,2019-02-20T13:15:00-06:00,"WESTLAKE VILLAGE, CA—Promising one lucky melon...",westlake village capromising one lucky melon f...
1885,Pope Francis Worried About Job Security After ...,2016-04-12T09:07:00-05:00,VATICAN CITY—Expressing his frustration with o...,vatican cityexpressing his frustration with on...
605,Hard To Tell If Wikipedia Entry On Dada Has Be...,2007-08-20T01:03:00-05:00,"ZURICH, SWITZERLAND—The Wikipedia entry on Dad...",zurich switzerlandthe wikipedia entry on dadat...


Extract just the content of each article and convert it to a list.

In [16]:
onioncontentlist = theonion.loc[:, 'processed'].values.tolist()
len(onioncontentlist)

342

In [17]:
oniontitlelist = theonion.loc[:, 'Title'].values.tolist()
oniontitlelist[:10]

['Nuclear-Bomb Instructions Found In Pentagon',
 'American Classmates Having Difficulty Understanding Better Educated Foreign Exchange Student',
 'Dole Reveals One Cantaloupe Out There Contains $10 Million Check',
 'Pope Francis Worried About Job Security After Butting Heads With New God',
 'Hard To Tell If Wikipedia Entry On Dada Has Been Vandalized Or Not',
 "Poll: 98% Of People Picture Run-Down Strip Mall Parking Lot When Word 'America' Said",
 'New Viacom Ad Tells Employees To Get Back To Work',
 'New Study Shows Majority Of Late Afternoon Sleepiness At Work Caused By Undetected Carbon Monoxide Leak',
 'Nation Could Really Use A Few Days Where It Isn’t Gripped By Something',
 'Cashier Allows Line-Cutting To Go Unpunished']

Compute the normalized term frequency using the function created [above](#normalized-term-frequency).

In [18]:
tf_onion = compute_normalizedtf(onioncontentlist)
tf_onion

[{'nuclear': 0.0125,
  'international': 0.0125,
  'nigel': 0.0125,
  'correspondent': 0.0125,
  'construction': 0.0125,
  'touring': 0.0125,
  'device': 0.0125,
  'while': 0.0125,
  'for': 0.025,
  'something': 0.0125,
  'development': 0.0125,
  'united': 0.0125,
  'came': 0.0125,
  'a': 0.05,
  'across': 0.0125,
  'working': 0.0125,
  'the': 0.0375,
  'sorts': 0.0125,
  'were': 0.0375,
  'all': 0.0125,
  'these': 0.0125,
  'desk': 0.0125,
  'of': 0.0375,
  'and': 0.025,
  'section': 0.0125,
  'definitely': 0.0125,
  'had': 0.0125,
  'on': 0.0125,
  'guys': 0.0125,
  'monday': 0.0125,
  'who': 0.0125,
  'arlington': 0.0125,
  'in': 0.0375,
  'compound': 0.0125,
  'manuals': 0.0125,
  'aftongeorge': 0.0125,
  'they': 0.025,
  'sept': 0.0125,
  'pentagon': 0.0125,
  'drawer': 0.0125,
  'an': 0.0125,
  'stuff': 0.0125,
  'its': 0.0125,
  'found': 0.0125,
  'do': 0.0125,
  'what': 0.0125,
  'va': 0.0125,
  'attack': 0.0125,
  'plans': 0.0375,
  'damaged': 0.0125,
  'think': 0.0125,
  'this

Compute the IDF using [compute_idf](#inverse-document-frequency-idf).

In [ ]:
idf_onion = compute_idf(onioncontentlist)

Pickle data so that computation does not need to be repeated.

In [19]:
import dill

In [ ]:
with open('./pickles/tf_onion.pkl', 'wb') as f:
    dill.dump(tf_onion, f)

with open('./pickles/idf_onion.pkl', 'wb') as f:
    dill.dump(idf_onion, f)

In [20]:
with open('./pickles/tf_onion.pkl', 'rb') as f:
    tf_onion = dill.load(f)

with open('./pickles/idf_onion.pkl', 'rb') as f:
    idf_onion = dill.load(f)

### Overload compute_tfidf_with_alldocs Function

I overloaded the `compute_tfidf_with_alldocs` function to allow for a term
frequency dictionary and IDF dictionary to be passed in.  This removes the
necessity for closures.

In [21]:
def compute_tfidf_with_alldocs(documents, query, tf_dict, idf_dict):
    tf_idf = []
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_dict[index]
        sentence = doc.split()
        for word in sentence:
            for text in query_tokens:
                if(text == word):
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf_idf , df

Create a new query for this new dataset and use the overloaded function to
produce the TF-IDF scores list and the dataframe of the TF-IDF scores for each
word in the query.

In [22]:
query = "clinton confirmed ecuador is a country"
tf_idf_onion , df_onion = compute_tfidf_with_alldocs(onioncontentlist, query, tf_onion, idf_onion)
print(df_onion)

       doc  clinton  confirmed  ecuador        is         a   country
0      0.0      0.0   0.000000      0.0  0.000000  0.052858  0.000000
1      1.0      0.0   0.000000      0.0  0.007034  0.004499  0.015387
2      2.0      0.0   0.000000      0.0  0.000000  0.054997  0.000000
3      3.0      0.0   0.000000      0.0  0.000000  0.018766  0.000000
4      4.0      0.0   0.000000      0.0  0.045601  0.029163  0.000000
..     ...      ...        ...      ...       ...       ...       ...
337  337.0      0.0   0.000000      0.0  0.008746  0.027967  0.000000
338  338.0      0.0   0.000000      0.0  0.006482  0.049749  0.000000
339  339.0      0.0   0.033636      0.0  0.009958  0.025474  0.021783
340  340.0      0.0   0.000000      0.0  0.009667  0.018547  0.021146
341  341.0      0.0   0.000000      0.0  0.000000  0.026429  0.000000

[342 rows x 7 columns]


Use the previous functions to calculate the term frequencies and IDF for
the query.

In [23]:
norm_tf_qry = compute_query_tf(query)
idf_dict_qry = compute_query_idf(query)
print(norm_tf_qry)
print(idf_dict_qry)

{'clinton': 0.16666666666666666, 'confirmed': 0.16666666666666666, 'ecuador': 0.16666666666666666, 'is': 0.16666666666666666, 'a': 0.16666666666666666, 'country': 0.16666666666666666}
{'clinton': 1.0, 'confirmed': 1.0, 'ecuador': 1.0, 'is': 1.0, 'a': 1.0, 'country': 1.0}


### Overload `compute_query_tfidf`

Once again, I overload the `compute_query_tfidf` function to allow the query
term frequency and IDF dictionaries to be passed in.

In [24]:
def compute_query_tfidf(query, norm_tf_qry, idf_dict_qry):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = norm_tf_qry[word] * idf_dict_qry[word]
    return tfidf_dict_qry

Run the function on the new query to find the TF-IDF for the query.

In [25]:
tfidf_dict_qry = compute_query_tfidf(query, norm_tf_qry, idf_dict_qry)
print(tfidf_dict_qry)

{'clinton': 0.16666666666666666, 'confirmed': 0.16666666666666666, 'ecuador': 0.16666666666666666, 'is': 0.16666666666666666, 'a': 0.16666666666666666, 'country': 0.16666666666666666}


In [26]:
def rank_similarity_docs(data, df, query):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist())
    return cos_sim

In [27]:
similarity_onion = rank_similarity_docs(onioncontentlist, df_onion, query)
similarity_onion = np.nan_to_num(np.array(similarity_onion).flatten())
theonion_cp = theonion.copy()
theonion_cp = theonion_cp.drop(["processed"], axis=1)
theonion_cp["similarity"] = similarity_onion

In [28]:
theonion_cp.sort_values(by="similarity", ascending=False)

,Title,Published Time,Content,similarity
4007,U.S. To Host Foster Country,2000-08-16T15:00:12-05:00,"WASHINGTON, DC–At a press conference Monday, P...",0.858234
4456,"Report: Oyster Cracker–Wise, Nation Doing Pret...",2015-09-14T12:10:00-05:00,WASHINGTON—Citing their ready availability and...,0.781624
4920,Kim Jong-Un Thrown Into Labor Camp For Attempt...,2018-04-27T12:38:00-05:00,"PYONGYANG—Following a swift capture, arrest, a...",0.764459
1922,Report: Trying To Hug Oncoming Train Still Lea...,2019-06-21T08:29:00-05:00,GENEVA—Calling the literal embrace of high-spe...,0.750952
4502,Nation’s Relatives Call For Little Zoom Tour O...,2020-11-26T08:00:00-06:00,"CARROLLTON, TX—Declaring “Ooh, yes” and “Let’s...",0.706028
...,...,...,...,...
3410,Houston Residents Begin Surveying Damage Of 20...,2017-08-28T15:32:00-05:00,HOUSTON—Appearing shellshocked as they took in...,0.000000
6842,Amsterdam Tourist Can't Find 'Kind Bud' In Phr...,2001-10-17T15:00:16-05:00,AMSTERDAM—While on vacation in Amsterdam Monda...,0.000000
5257,MC Serch Updates List Of Gas-Face Recipients,2003-06-11T15:00:10-05:00,"QUEENS, NY—For the first time since the list's...",0.000000
2181,Man Adds A Few Personalized Tracks To Standard...,2003-05-21T15:00:16-05:00,"SPRINGFIELD, MO—Wanting to add something speci...",0.000000


In [223]:
theonion_cp.loc[4007, "Content"]

'WASHINGTON, DC–At a press conference Monday, President Clinton confirmed that the U.S. is clearing out a portion of Montana to make room for foster country Ecuador. "Ecuador has been through some pretty rough times these last few years, bounced around from one foster homeland to another," Clinton said of the troubled South American nation, which lost its government in a March 1996 earthquake. "But it\'s a tough little nation, and with a lot of love and a little political stability, it\'s going to be just fine." Ecuador\'s previous host, Denmark, returned the country after just three weeks, complaining that it consumed too much of its food and petroleum.'

In [224]:
theonion_cp.loc[4456, "Content"]

'WASHINGTON—Citing their ready availability and consistent quality, a report released Monday by the Brookings Institution confirmed that, as far as oyster crackers go, the nation is doing pretty good. “The United States is currently in a very respectable place in terms of oyster crackers, and at present, any existing oyster cracker–related concerns are minimal,” said the report’s lead researcher, Kevin Purcell, who offered the prevalence of oyster crackers in supermarkets, the rarity with which they are discovered broken, and the fact that packets of the crackers—often two at a time—are handed out free of charge with many soups and chowders as clear evidence that the country is in a solid spot, oyster cracker–wise. “Using these small crackers as our sole metric, the findings could not be any clearer: We are doing well as a nation. Our needs for oyster crackers are being met and then some. Could things take a downturn in the future vis-a-vis oyster crackers? Certainly. But there is noth

## Program_2

Use only term frequency to compute similarities.

In [32]:
def compute_tf_with_alldocs_p2(documents, query, tf_dict):
    tf = []
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_dict[index]
        sentence = doc.split()
        for word in sentence:
            for text in query_tokens:
                if(text == word):
                    idx = sentence.index(word)
                    tf_score = doc_num[word]
                    tf.append(tf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf , df

In [33]:
tf_onion_p2 , df_onion_p2 = compute_tf_with_alldocs_p2(onioncontentlist, query, tf_onion)
print(df_onion_p2)

       doc  exercise      more  pain
0      0.0       0.0  0.000000   0.0
1      1.0       0.0  0.004255   0.0
2      2.0       0.0  0.000000   0.0
3      3.0       0.0  0.000000   0.0
4      4.0       0.0  0.000000   0.0
..     ...       ...       ...   ...
337  337.0       0.0  0.000000   0.0
338  338.0       0.0  0.000000   0.0
339  339.0       0.0  0.000000   0.0
340  340.0       0.0  0.005848   0.0
341  341.0       0.0  0.000000   0.0

[342 rows x 4 columns]


In [34]:
def compute_query_tf_p2(query, norm_tf_qry):
    tf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tf_dict_qry[word] = norm_tf_qry[word]
    return tf_dict_qry

In [35]:
tf_dict_qry = compute_query_tf_p2(query, norm_tf_qry)
print(tf_dict_qry)

{'exercise': 0.3333333333333333, 'more': 0.3333333333333333, 'pain': 0.3333333333333333}


In [36]:
similarity_onion_p2 = rank_similarity_docs(onioncontentlist, df_onion_p2, query)
similarity_onion_p2 = np.nan_to_num(np.array(similarity_onion_p2).flatten())
theonion_cp_p2 = theonion.copy()
theonion_cp_p2 = theonion_cp_p2.drop(["processed"], axis=1)
theonion_cp_p2["similarity"] = similarity_onion_p2

In [37]:
theonion_cp_p2.sort_values(by="similarity", ascending=False)

,Title,Published Time,Content,similarity
143,New Study Finds Running For 20 Minutes Each Da...,2014-07-31T09:58:00-05:00,"EAST LANSING, MI—According to a study released...",0.962250
4715,Beyond Meat Researchers Announce Creation Of F...,2019-04-17T14:58:00-05:00,"EL SEGUNDO, CA—As part of the company’s ongoin...",0.816497
3677,Frito-Lay Pledges Party-Size Bag Of Plain Pota...,2020-01-31T11:55:00-06:00,"PLANO, TX—Citing an ongoing commitment to its ...",0.577350
1173,Facebook Bans Extremist Figures After Designat...,2019-05-02T16:00:00-05:00,"MENLO PARK, CA—In an effort to finally crack d...",0.577350
1938,Instagram Assures Parents They’ve Hired Dozens...,2021-09-27T12:35:00-05:00,MENLO PARK—Assuring parents that the crew were...,0.577350
...,...,...,...,...
28,Obama Resigns From Presidency After Michelle L...,2016-02-10T11:34:00-06:00,‘It’s Time I Made Some Sacrifices For This Fam...,0.000000
470,New Partially Digested Doritos Eliminate Tedio...,2000-05-17T15:00:15-05:00,"DALLAS–At a press conference Monday, Frito-Lay...",0.000000
3624,Rats Scramble To Hide Fully Functioning Amusem...,2021-04-08T07:00:00-05:00,CHICAGO—Squeaking wildly to one another as the...,0.000000
4292,Study Finds Humans Evolved Fingers To Stop Dro...,2018-09-28T09:39:00-05:00,CHICAGO—In what many have deemed a “missing li...,0.000000


In [38]:
theonion_cp_p2.loc[4007, "Content"]

'WASHINGTON, DC–At a press conference Monday, President Clinton confirmed that the U.S. is clearing out a portion of Montana to make room for foster country Ecuador. "Ecuador has been through some pretty rough times these last few years, bounced around from one foster homeland to another," Clinton said of the troubled South American nation, which lost its government in a March 1996 earthquake. "But it\'s a tough little nation, and with a lot of love and a little political stability, it\'s going to be just fine." Ecuador\'s previous host, Denmark, returned the country after just three weeks, complaining that it consumed too much of its food and petroleum.'

In [39]:
theonion_cp_p2.loc[5592, "Content"]

'EAST HARTFORD, CT—Thinking wearily of the moment when he would have to return to the daily grind, local man Dan Zageris is already dreading going back to his marriage Monday, sources confirmed this weekend. “A couple of days away are great, but I know that when Monday morning rolls around, it will just start up all over again,” said Zageris, 36, adding that his anxiety will likely return on Sunday, as he becomes preoccupied with the thought of heading back to his monogamous relationship of eight years the following day. “Time off always flies by, and before I know it, I’m clocking back in with my wife and have another long slog of marriage ahead of me.” At press time, Zageris was reportedly keeping his head down and hoping to power through the next 40 years or so.'

## Compare P1 and P2

In [51]:
def rank_similarity_docs(data, df, query, dict_qry):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(dict_qry, df , query , doc_num).tolist())
    return cos_sim

In [92]:
queries = ["clinton confirmed ecuador is a country", "it is burning the world a lot", "exercise more pain"]

theonion_tfidf = theonion.copy()
theonion_tfidf = theonion_tfidf.drop(["processed"], axis=1)
for query in queries:
    tf_idf_onion , df_onion = compute_tfidf_with_alldocs(onioncontentlist, query, tf_onion, idf_onion)
    norm_tf_qry = compute_query_tf(query)
    idf_dict_qry = compute_query_idf(query)
    tfidf_dict_qry = compute_query_tfidf(query, norm_tf_qry, idf_dict_qry)
    similarity_onion = rank_similarity_docs(onioncontentlist, df_onion, query, tfidf_dict_qry)
    similarity_onion = np.nan_to_num(np.array(similarity_onion).flatten())
    theonion_tfidf[query] = similarity_onion
    print(theonion_tfidf.sort_values(by=query, ascending=False).head())

                                                  Title  \
4007                        U.S. To Host Foster Country   
4456  Report: Oyster Cracker–Wise, Nation Doing Pret...   
4920  Kim Jong-Un Thrown Into Labor Camp For Attempt...   
1922  Report: Trying To Hug Oncoming Train Still Lea...   
4502  Nation’s Relatives Call For Little Zoom Tour O...   

                 Published Time  \
4007  2000-08-16T15:00:12-05:00   
4456  2015-09-14T12:10:00-05:00   
4920  2018-04-27T12:38:00-05:00   
1922  2019-06-21T08:29:00-05:00   
4502  2020-11-26T08:00:00-06:00   

                                                Content  \
4007  WASHINGTON, DC–At a press conference Monday, P...   
4456  WASHINGTON—Citing their ready availability and...   
4920  PYONGYANG—Following a swift capture, arrest, a...   
1922  GENEVA—Calling the literal embrace of high-spe...   
4502  CARROLLTON, TX—Declaring “Ooh, yes” and “Let’s...   

      clinton confirmed ecuador is a country  
4007                            

In [93]:
theonion_tfonly = theonion.copy()
theonion_tfonly = theonion_tfonly.drop(["processed"], axis=1)
for query in queries:
    tf_onion_p2 , df_onion_p2 = compute_tf_with_alldocs_p2(onioncontentlist, query, tf_onion)
    norm_tf_qry = compute_query_tf(query)
    tf_dict_qry = compute_query_tf_p2(query, norm_tf_qry)
    similarity_onion_p2 = rank_similarity_docs(onioncontentlist, df_onion_p2, query, tf_dict_qry)
    similarity_onion_p2 = np.nan_to_num(np.array(similarity_onion_p2).flatten())
    theonion_tfonly[query] = similarity_onion_p2
    print(theonion_tfonly.sort_values(by=query, ascending=False).head())

                                                  Title  \
4007                        U.S. To Host Foster Country   
5592  Marriage Going To Be Hard To Go Back To On Monday   
4100          Landlord Not Convinced Heat Isn't Working   
330   Man On Weird Fad Diet Where He Eats Flavorful ...   
4611  American Classmates Having Difficulty Understa...   

                 Published Time  \
4007  2000-08-16T15:00:12-05:00   
5592  2014-07-18T10:20:00-05:00   
4100  2008-02-08T00:00:22-06:00   
330   2015-01-15T08:00:00-06:00   
4611  2018-08-23T10:14:00-05:00   

                                                Content  \
4007  WASHINGTON, DC–At a press conference Monday, P...   
5592  EAST HARTFORD, CT—Thinking wearily of the mome...   
4100  QUEENS, NY—Despite urgent pleas to the contrar...   
330   MARIN, CA—Admitting that the odd lifestyle cha...   
4611  SACRAMENTO, CA—Addressing the glaringly obviou...   

      clinton confirmed ecuador is a country  
4007                            

In [94]:
tfidf_top5 = [theonion_tfidf.sort_values(by=query, ascending=False).drop(theonion_tfidf.columns.difference(["Title", "Content", query]), axis=1).head() for query in queries]
tf_top5 = [theonion_tfonly.sort_values(by=query, ascending=False).drop(theonion_tfonly.columns.difference(["Title", "Content", query]), axis=1).head() for query in queries]

### Queries

#### "clinton confirmed ecuador is a country"

In [95]:
tfidf_top5[0]

,Title,Content,clinton confirmed ecuador is a country
4007,U.S. To Host Foster Country,"WASHINGTON, DC–At a press conference Monday, P...",0.858234
4456,"Report: Oyster Cracker–Wise, Nation Doing Pret...",WASHINGTON—Citing their ready availability and...,0.781624
4920,Kim Jong-Un Thrown Into Labor Camp For Attempt...,"PYONGYANG—Following a swift capture, arrest, a...",0.764459
1922,Report: Trying To Hug Oncoming Train Still Lea...,GENEVA—Calling the literal embrace of high-spe...,0.750952
4502,Nation’s Relatives Call For Little Zoom Tour O...,"CARROLLTON, TX—Declaring “Ooh, yes” and “Let’s...",0.706028


In [96]:
tf_top5[0]

,Title,Content,clinton confirmed ecuador is a country
4007,U.S. To Host Foster Country,"WASHINGTON, DC–At a press conference Monday, P...",0.808290
5592,Marriage Going To Be Hard To Go Back To On Monday,"EAST HARTFORD, CT—Thinking wearily of the mome...",0.707107
4100,Landlord Not Convinced Heat Isn't Working,"QUEENS, NY—Despite urgent pleas to the contrar...",0.707107
330,Man On Weird Fad Diet Where He Eats Flavorful ...,"MARIN, CA—Admitting that the odd lifestyle cha...",0.707107
4611,American Classmates Having Difficulty Understa...,"SACRAMENTO, CA—Addressing the glaringly obviou...",0.707107


#### "it is burning the world a lot"

In [97]:
tfidf_top5[1]

,Title,Content,it is burning the world a lot
3677,Frito-Lay Pledges Party-Size Bag Of Plain Pota...,"PLANO, TX—Citing an ongoing commitment to its ...",0.808085
1888,Nation Finally Stands Far Enough Away From Jac...,NEW YORK—Straining their eyes from the opposit...,0.803051
769,Wedding Planner Suggests Replacing Unsightly G...,"NASHVILLE, TN—Saying that a lot of brides don’...",0.799797
1294,Depressed Gallup Director Issues Poll Asking W...,WASHINGTON—In an effort to accurately gauge Am...,0.798101
867,Report: Most Terrorists Do Not Start The Day O...,"GENEVA, SWITZERLAND–In a report that is prompt...",0.780074


In [98]:
tfidf_top5[1].loc[3677, "Content"]

'PLANO, TX—Citing an ongoing commitment to its community and the world at large, international snack food giant Frito-Lay held a major press conference Friday to announce plans to donate a party-size bag of regular, unflavored potato chips to the fight against global hunger. “As responsible corporate citizens, we believe it is our duty to take direct action and contribute one 15.25-ounce bag of Lay’s Classic potato chips to the world’s food-insecure population,” said company spokesperson Mark Walters, unveiling a 10-year plan to distribute the single bag of chips to those famine-stricken regions of the world where it is most needed. “With this donation, we are pledging approximately 225 potato chips that, when taken together, will provide a grand total of nearly 2,500 calories to malnourished and snack-deprived men, women, and children across the globe. What’s more, countries willing to appropriate matching government funds for this new program will be eligible to receive up to 2 table

In [99]:
tf_top5[1]

,Title,Content,it is burning the world a lot
5347,If Area Dad Steps On Legos One More Time,"DARLINGTON, SC—According to loud reports from ...",0.742307
1294,Depressed Gallup Director Issues Poll Asking W...,WASHINGTON—In an effort to accurately gauge Am...,0.741941
3677,Frito-Lay Pledges Party-Size Bag Of Plain Pota...,"PLANO, TX—Citing an ongoing commitment to its ...",0.733625
1888,Nation Finally Stands Far Enough Away From Jac...,NEW YORK—Straining their eyes from the opposit...,0.729293
898,Lab Mouse Nervous For First Day Of New Job Get...,"STANFORD, CA—Anxious about all the brand-new c...",0.722914


In [102]:
tf_top5[1].loc[5347, "Content"]

'DARLINGTON, SC—According to loud reports from within the Kaminsky household Tuesday, if area father Russell Kaminsky steps on one more goddamn Lego, man, forget about it. "Gaaaaaaaaaa!" shouted Kaminsky, grimacing as he extracted a blue, two-peg Lego brick from his right instep. "I\'ve told you a hundred times. This is it, this is the last warning: I step on one more Lego, and no one will ever step on another Lego in this house ever again, I promise." Observers are questioning Kaminsky\'s willingness to actually follow through on the threat, citing his failure to deliver on his Lincoln Logs ultimatum of last March. '

#### "exercise more pain"

In [ ]:
tfidf_top5[2]

,Title,Content,exercise more pain
143,New Study Finds Running For 20 Minutes Each Da...,"EAST LANSING, MI—According to a study released...",0.935228
4715,Beyond Meat Researchers Announce Creation Of F...,"EL SEGUNDO, CA—As part of the company’s ongoin...",0.749311
3437,Celebrity Disappointed After Meeting Fan,"LOS ANGELES— Denzel Washington, who on Monday ...",0.577350
3028,Nation's Deans Meet To Discuss Problem Of Coll...,"GAINESVILLE, FL—Calling the trend ""a black mar...",0.577350
2149,Study Finds Owning Cool Leather Jacket More Re...,SEATTLE—A study released Thursday by the Pacif...,0.577350


In [101]:
tf_top5[2]

,Title,Content,exercise more pain
143,New Study Finds Running For 20 Minutes Each Da...,"EAST LANSING, MI—According to a study released...",0.962250
4715,Beyond Meat Researchers Announce Creation Of F...,"EL SEGUNDO, CA—As part of the company’s ongoin...",0.816497
3677,Frito-Lay Pledges Party-Size Bag Of Plain Pota...,"PLANO, TX—Citing an ongoing commitment to its ...",0.577350
1173,Facebook Bans Extremist Figures After Designat...,"MENLO PARK, CA—In an effort to finally crack d...",0.577350
1938,Instagram Assures Parents They’ve Hired Dozens...,MENLO PARK—Assuring parents that the crew were...,0.577350


## Writeup

Beyond some shuffling between the articles, we find that search results were
very similar between using TF-IDF and TF to calculate cosine similarities.
Nevertheless, we can see that there are nuances between them.  First, with
careful inspection, we see that the TF representation of the word vectors put
more weight on stop words such as *is*, *the*, *a*, etc.  For the query "it is
burning the world a lot", notice that the top search result for the TF-IDF put
much more weight on the word "world", whereas the TF result contained no words
matching *burning*, *world*, or *lot*. Also notice that when there were large
numbers of matches with rarer words, the TF-IDF exhibited more confidence with
the match (higher cosine similarity scores).  The first query contained
*clinton*, *confirmed*, *ecuador*, and *country*.  Although the top match was
identical for both strategies, the TF-IDF had an increased similarity score over
the TF representation.